## More Transcribe Python samples can be found here:
https://docs.aws.amazon.com/code-samples/latest/catalog/code-catalog-python-example_code-transcribe.html

In [ ]:
"""
Purpose

Shows how to use AWS SDK for Python (Boto3) to call Amazon Transcribe to make a
transcription of an audio file.

This script is intended to be used with the instructions for getting started in the
Amazon Transcribe Developer Guide here:
    https://docs.aws.amazon.com/transcribe/latest/dg/getting-started-python.html.
"""

import time
import boto3

from urllib.request import urlopen
import json

In [ ]:
def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='mp3',
        LanguageCode='en-US'
    )
    transcript = ''
    data = ''
    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED': 
                response = urlopen(job['TranscriptionJob']['Transcript']['TranscriptFileUri'])
                data = json.loads(response.read())
                transcript = data['results']['transcripts'][0]['transcript']    
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)
    return (transcript, data)

In [ ]:
## remember to use your own bucket and mp3 file
transcribe_client = boto3.client('transcribe', 'us-east-1')
file_uri = 's3://noureddin-test-bucket/welcome.mp3'
(transcript,data) = transcribe_file('Transcribe-job-123', file_uri, transcribe_client)
print(transcript)

In [ ]:
data